<a href="https://colab.research.google.com/github/helincesxyz/finetuneLLM/blob/main/Copy_of_bonito_data_instruction_generartion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@ Created by Yanli LIU

This demo aim to show a much more cost-efficient alternative to automate the creation of instruction datasets from any documents, using a lightweight open-source library called "Bonito".

## Step 1 - Install the dependencies

In [ ]:
!pip install -e git+https://github.com/BatsResearch/bonito#egg=bonito

Obtaining bonito from git+https://github.com/BatsResearch/bonito#egg=bonito
  Cloning https://github.com/BatsResearch/bonito to ./src/bonito
  Running command git clone --filter=blob:none --quiet https://github.com/BatsResearch/bonito /content/src/bonito
  Resolved https://github.com/BatsResearch/bonito to commit 0b6b23ddc1e0aaeefcd9072352e7166a281f5706
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets huggingface_hub

In [ ]:
!pip install pymupdf spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 30.1 MB/s eta 0:00:00


## Step 2: Processing the PDF documentExtract Text from PDF

### 2.1 Exract texts

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = 'cssf12_552eng.pdf'
text = extract_text_from_pdf(pdf_path)


FileNotFoundError: no such file: 'cssf12_552eng.pdf'

### 2.2 Split Text into Sentences

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Load English tokenizer, tagger, parser, NER, and word vectors

def split_into_sentences(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

sentences = split_into_sentences(text)


In [ ]:
print(len(sentences))

1121


In [ ]:
print(sentences[500])

The crisis management arrangements shall be tested and updated, on a 
regular basis, in order to ensure and maintain its effectiveness.


### 2.3 Create a Transformers Dataset
You'll now transform the sentences into a format suitable for the Hugging Face datasets library.

In [ ]:
from datasets import Dataset

# Assuming sentences is a list of strings, where each string is a sentence
data = {"sentence": sentences}
dataset = Dataset.from_dict(data)

print(dataset)


Dataset({
    features: ['sentence'],
    num_rows: 1121
})


## Step 3 : Generate synthetic dataset using Bonito

In [ ]:
from bonito import Bonito, SamplingParams
from datasets import load_dataset

# Initialize the Bonito model
bonito = Bonito("BatsResearch/bonito-v1")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

INFO 03-03 20:22:44 llm_engine.py:87] Initializing an LLM engine with config: model='BatsResearch/bonito-v1', tokenizer='BatsResearch/bonito-v1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

INFO 03-03 20:22:50 weight_utils.py:163] Using model weights format ['*.bin']


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

INFO 03-03 20:23:54 llm_engine.py:357] # GPU blocks: 9133, # CPU blocks: 2048
INFO 03-03 20:23:56 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-03 20:23:56 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-03 20:24:04 model_runner.py:756] Graph capturing finished in 8 secs.


In [ ]:
# load dataset with unannotated text
# Supported Task Types [full name (short form)]: extractive question answering (exqa),
# multiple-choice question answering (mcqa),
# question generation (qg),
# question answering without choices (qa),
# yes-no question answering (ynqa),
# coreference resolution (coref),
# paraphrase generation (paraphrase),
# paraphrase identification (paraphrase_id),
# sentence completion (sent_comp),
# sentiment (sentiment),
# summarization (summarization),
# text generation (text_gen),
# topic classification (topic_class),
# word sense disambiguation (wsd), textual entailment (te), natural language inference (nli)
# Generate synthetic instruction tuning dataset

sampling_params = SamplingParams(max_tokens=256, top_p=0.95, temperature=0.5, n=1)
synthetic_dataset = bonito.generate_tasks(
    dataset,
    context_col="sentence",
    task_type="qg",
    sampling_params=sampling_params
)

Map:   0%|          | 0/1121 [00:00<?, ? examples/s]

Processed prompts: 100%|██████████| 1121/1121 [00:27<00:00, 41.02it/s]


Filter:   0%|          | 0/1121 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

In [ ]:
print(synthetic_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 1120
})


In [ ]:
import pandas as pd

df = pd.DataFrame(synthetic_dataset)

print(df.head(50))  # Adjust the number inside head() to see more or fewer rows


                                                input  \
0   Write a multi-choice question for the followin...   
1   Write a multi-choice question for the followin...   
2   No 15-02 relating to the supervisory review \n...   
3   I want to test the ability of students to read...   
4   It reflects the European and \ninternational p...   
5   Based on the context and the answer, generate ...   
6   This concerns especially \nthe European Bankin...   
7   The principles and good practices arising from...   
8   1 To professionals performing lending operatio...   
9   on the \nfinancial sector, Chapter 3 of Part I...   
10  Chapter 2(12) of Part III shall also apply.\n\...   
11  Write a multi-choice question for the followin...   
12  As regards the appointments of members of the ...   
13  The Circular is divided into four parts: the f...   
14  Write a multi-choice question for the followin...   
15  Write a multi-choice question for the followin...   
16  Write a multi-choice questi

## Step 4 : Saving the generated dataset

4.1Authenticate with Hugging Face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

4.2 Push the dataset to the Hub

In [ ]:
from huggingface_hub import create_repo
from huggingface_hub import Repository

repo_name = "dataset_12_552"  # Choose a name for your dataset repository
repo_url = create_repo(repo_name, repo_type="dataset")
print("Repository URL:", repo_url)






Repository URL: https://huggingface.co/datasets/Ronal999/dataset_12_552


In [ ]:
synthetic_dataset.push_to_hub(f"Ronal999/dataset_12_552")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ronal999/dataset_12_552/commit/a792ef9cf12babd2f3570292282072b89c99dc6d', commit_message='Upload dataset', commit_description='', oid='a792ef9cf12babd2f3570292282072b89c99dc6d', pr_url=None, pr_revision=None, pr_num=None)